## Word2Vec 실습

### 학습

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gensim konlpy

In [ ]:
from gensim.models import Word2Vec
from konlpy.tag import Okt

In [ ]:
# 샘플 문장
sample_sentences = ["안녕하세요 저는 학생입니다", "파이썬은 매우 재미있는 프로그래밍 언어입니다", "자연어 처리는 흥미로운 분야입니다"]

In [ ]:
# 토큰화
okt = Okt()
sentences_tokenized = [okt.morphs(sentence) for sentence in sample_sentences]

In [ ]:
# Word2Vec 모델 학습
model = Word2Vec(sentences_tokenized, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# 단어 벡터 얻기
vector = model.wv['안녕하세요']
print(vector)

In [ ]:
# 단어 사전 수
len(model.wv)

15

In [ ]:
# 가장 유사한 단어
model.wv.most_similar('학생')

[('처리', 0.15016666054725647),
 ('은', 0.12813611328601837),
 ('입니다', 0.0931011214852333),
 ('안녕하세요', 0.09215975552797318),
 ('분야', 0.04652618244290352),
 ('언어', 0.0007307013729587197),
 ('는', -0.003640753449872136),
 ('흥미로운', -0.009253410622477531),
 ('파이썬', -0.03030233457684517),
 ('재미있는', -0.04044129699468613)]

In [ ]:
# 저장
model.save("word2vec_model.bin")

### 저장한 Word2Vec 모델을 불러와 임베딩 값을 얻는 방법

In [ ]:
from gensim.models import Word2Vec
import numpy as np
from scipy.spatial.distance import cosine

In [ ]:
# 사전 훈련된 모델 로드
model = Word2Vec.load("word2vec_model.bin")

In [ ]:
# KoNLPy의 Okt를 사용하여 한국어 텍스트 토큰화
okt = Okt()

In [ ]:
# 문장을 벡터로 변환하는 함수
def sentence_vector(sentence, model):
  # 토큰화
  words = okt.morphs(sentence)

  # 단어 벡터화
  # 단어가 모델에 있을 경우 벡터를 추출
  word_vectors = [model.wv[word] for word in words if word in model.wv]

  # 벡터의 유사도를 구하기 위해 각 단어마다 나온 값들에 대한 평균을 계산
  if not word_vectors:
      return np.zeros(model.vector_size)

  return np.mean(word_vectors, axis=0)

In [ ]:
# 예시 문장들
sentence1 = "저는 학생입니다."
sentence2 = "나는 학생입니다."

In [ ]:
# 문장 벡터화
vec1 = sentence_vector(sentence1, model)
vec2 = sentence_vector(sentence2, model)

In [ ]:
print(cosine(vec1, vec2)) # 코사인 거리
print(1 - cosine(vec1, vec2)) # 코사인 유사도

0.13259774446487427
0.8674022555351257


## 임베딩 실습 1
### 활용 데이터: sample.json

### 실습 개요
이 실습에서는 사전 훈련된 모델 로드하여 주어진 csv 파일에 포함된 텍스트 데이터에 대해 Okt 형태소 분석기를 사용하여 토큰화를 수행하고, 그 결과를 기반으로 Word2Vec 모델 학습하는 실습입니다. KoNLPy 라이브러리 Okt 형태소 분석기를 활용하여 토큰화를 수행하고, 토큰화된 데이터를 활용해 Word2Vec모델을 학습한 다음 저장합니다.


### 실습 절차
1. 라이브러리 설치 및 임포트: 필요한 라이브러리 (gensim, konlpy)를 설치하고 임포트합니다.
2. sample.json 데이터를 pandas를 활용해 읽습니다.
3. src_sentence를 활용해 word2vec 모델을 만듭니다. (이때, Okt 형태소 분석기를 활용하여 토큰화를 수행합니다.)
4. 만든 모델을 저장합니다.


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install konlpy gensim

In [30]:
import json
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
from scipy.spatial.distance import cosine
from konlpy.tag import Okt

In [4]:
with open('/content/drive/MyDrive/NLP 실습/1 11/sample.json') as f:
  data = json.load(f)

In [5]:
len(data)

2

In [9]:
data.keys()

dict_keys(['chunk-number', 'paragraph'])

In [11]:
len(data['paragraph'])

2

In [14]:
data['paragraph'][0].keys()

dict_keys(['info', 'sentences'])

In [18]:
print(len(data['paragraph'][0]['sentences']))
print(len(data['paragraph'][1]['sentences']))

651
104


In [19]:
df1 = pd.DataFrame(data['paragraph'][0]['sentences'])
df2 = pd.DataFrame(data['paragraph'][1]['sentences'])

In [20]:
df1.head(2)

,src_sentence,tgt_sentence,src_lang,tgt_lang,src_paragraphs_id,src_word_count,spc_technical_label,spc_idiomatic_label,spc_proper_label,spc_double_label
0,1990년대와 2000년대는 마케팅 분야에서 브랜드관리가 매우 부상했다.,The 1990s and 2000s saw the rise of brand mana...,ko,en,SI-2210121323P16603884R0200215,7,"마케팅,브랜드관리",None,None,None
1,이렇게 커피 시장이 커지고 경쟁이 치열해지다 보니 커피 업체의 마케팅도 치열하다.,As the coffee market grows and competition int...,ko,en,SI-2210121323P16603884R0200373,11,"커피,시장,경쟁,마케팅,커피",None,None,None


In [21]:
df2.head(2)

,src_sentence,tgt_sentence,src_lang,tgt_lang,src_paragraphs_id,src_word_count,spc_technical_label,spc_idiomatic_label,spc_proper_label,spc_double_label
0,애경과 현대카드에서 마케팅팀장을 역임한 저자는 서울시 도시마케팅 전담기구로 출범한 ...,"As the head of Seoul Tourism Marketing, which ...",ko,en,SI-2210121323P07104842R0200003,20,"마케팅,역임,저자,시,도시,마케팅,기구,관광마케팅,세계,가슴,도시,각인,마케팅,총괄",None,"서울,서울,서울",None
1,현대경영에서 마케팅의 중요성을 강조하는 것도 이미 진부한 일이다.,It is now banal to emphasize the importance of...,ko,en,SI-2210121323P07104842R0200006,8,"경영,마케팅,강조,일",None,None,None


In [22]:
df = pd.concat([df1, df2])

In [23]:
df.head(1)

,src_sentence,tgt_sentence,src_lang,tgt_lang,src_paragraphs_id,src_word_count,spc_technical_label,spc_idiomatic_label,spc_proper_label,spc_double_label
0,1990년대와 2000년대는 마케팅 분야에서 브랜드관리가 매우 부상했다.,The 1990s and 2000s saw the rise of brand mana...,ko,en,SI-2210121323P16603884R0200215,7,"마케팅,브랜드관리",None,None,None


In [24]:
len(df)

755

In [34]:
sample_sentences = df['src_sentence'].values

In [35]:
# 토큰화
okt = Okt()
sentences_tokenized = [okt.morphs(sentence) for sentence in sample_sentences]

In [36]:
# Word2Vec 모델 학습
model = Word2Vec(sentences_tokenized, vector_size=100, window=5, min_count=1, workers=4)

In [40]:
# 가장 유사한 단어
model.wv.most_similar('경영')

[('서비스', 0.9828951358795166),
 ('통해', 0.982687771320343),
 ('마케팅', 0.9825984835624695),
 ('고객', 0.981902003288269),
 ('에서', 0.9818128943443298),
 ('커피', 0.981644332408905),
 ('하고', 0.9816119074821472),
 ('된', 0.9815881848335266),
 ('에', 0.9815772771835327),
 ('이', 0.9815609455108643)]

In [37]:
# 저장
model.save("word2vec_model.bin")

## Doc2Vec 실습

### 학습

In [41]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Okt

In [42]:
# 샘플 문장
documents = ["안녕하세요 저는 학생입니다", "파이썬은 매우 재미있는 프로그래밍 언어입니다", "자연어 처리는 흥미로운 분야입니다"]

# 토큰화
okt = Okt()

In [43]:
# 태그가 붙은 문서 생성
tagged_data = [TaggedDocument(words=okt.morphs(_d), tags=[str(i)]) for i, _d in enumerate(documents)]

In [44]:
tagged_data

[TaggedDocument(words=['안녕하세요', '저', '는', '학생', '입니다'], tags=['0']),
 TaggedDocument(words=['파이썬', '은', '매우', '재미있는', '프로그래밍', '언어', '입니다'], tags=['1']),
 TaggedDocument(words=['자연어', '처리', '는', '흥미로운', '분야', '입니다'], tags=['2'])]

In [45]:
# Doc2Vec 모델 초기화 및 훈련
model = Doc2Vec(vector_size=40, window=2, min_count=1, workers=4, epochs=40)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [46]:
# 문서 벡터 얻기
kor_vector = model.infer_vector(okt.morphs("안녕하세요."))
print("Korean Vector:", kor_vector)

Korean Vector: [ 0.00145777 -0.00665023 -0.00181094  0.00031811  0.012112   -0.0042143
 -0.00110593  0.01172749 -0.00536724 -0.00039846  0.00142583  0.00713892
  0.00544711  0.01112577  0.00126862 -0.00970586  0.01237327 -0.00427112
 -0.00967907  0.00593927 -0.00675014  0.01115226 -0.01089625  0.00560086
  0.00071247  0.00290209  0.00397291 -0.00464673  0.00161154  0.00498213
  0.00298031  0.00726688 -0.00056207 -0.00192532  0.00150525 -0.00839468
  0.00284788 -0.0073485  -0.00258266  0.00018508]


In [53]:
doc_vector = model.infer_vector(okt.morphs("안녕하세요."))

# 가장 유사한 문서 찾기
similar_docs = model.docvecs.most_similar([doc_vector], topn=5)

for doc_id, similarity in similar_docs:
    print(f"Document ID: {doc_id}, Similarity: {similarity}")

Document ID: 0, Similarity: 0.3715185523033142
Document ID: 1, Similarity: 0.13028830289840698
Document ID: 2, Similarity: -0.07718368619680405


<ipython-input-53-53731efe6b88>:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_docs = model.docvecs.most_similar([doc_vector], topn=5)


In [54]:
model.save("doc2vec_model.bin")

### 저장한 Doc2Vec 모델을 불러와 임베딩 값을 얻는 방법

In [55]:
from gensim.models.doc2vec import Doc2Vec
import numpy as np
from scipy.spatial.distance import cosine

In [57]:
# 로드
model = Doc2Vec.load("doc2vec_model.bin")

In [69]:
# 문서 벡터 얻기
vec1 = model.infer_vector(okt.morphs("이 문서는 학생에 관련된 문서입니다."))
vec2 = model.infer_vector(okt.morphs("학생에 관한 문서 입니다."))

In [70]:
print(cosine(vec1, vec2)) # 코사인 거리
print(1 - cosine(vec1, vec2)) # 코사인 유사도

0.8766956776380539
0.1233043223619461


## 임베딩 실습 2
### 활용 데이터: sample.json

### 실습 개요
실습 1의 과정을 Doc2Vec으로 변경하여 수행합니다.



In [71]:
import json
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
import numpy as np
from scipy.spatial.distance import cosine
from konlpy.tag import Okt

In [72]:
with open('/content/drive/MyDrive/NLP 실습/1 11/sample.json') as f:
  data = json.load(f)

In [73]:
df1 = pd.DataFrame(data['paragraph'][0]['sentences'])
df2 = pd.DataFrame(data['paragraph'][1]['sentences'])

In [74]:
df = pd.concat([df1, df2])

In [75]:
# 샘플 문장
documents = df['src_sentence'].values

# 토큰화
okt = Okt()

In [76]:
# 태그가 붙은 문서 생성
tagged_data = [TaggedDocument(words=okt.morphs(_d), tags=[str(i)]) for i, _d in enumerate(documents)]

In [77]:
# Doc2Vec 모델 초기화 및 훈련
model = Doc2Vec(vector_size=40, window=2, min_count=1, workers=4, epochs=40)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [78]:
# 문서 벡터 얻기
kor_vector = model.infer_vector(okt.morphs("안녕하세요."))
print("Korean Vector:", kor_vector)

Korean Vector: [-0.00223009 -0.01695544  0.02789298  0.02463179  0.01429797  0.02622431
  0.00394954 -0.00898418 -0.02193863  0.02867102  0.03996652 -0.04361594
  0.01045868  0.01621829  0.00187857  0.01120721  0.03470557  0.02330012
 -0.01172534 -0.03778464  0.0039556   0.014101    0.02375777  0.0198179
 -0.01655313  0.02117481  0.00017481  0.0016008  -0.00147241 -0.01853108
  0.05090566  0.01471231  0.01904207  0.00275242 -0.01935877  0.02914805
  0.02089044 -0.05941051 -0.05633527 -0.03943633]


In [ ]:
model.save("doc2vec_model.bin")